# <center> Procesamiento de habla para Learning Machine:</center>
# Mel-Frecuency Ceptral Coefficients (MFCCs)

El reconocimiento de habla  hoy en día es una herramienta muy utilizada para la interacción con la tecnología en general, desde el ámbito de entretenimiento al ámbito profesional.

Metodología:
Para que una máquina logre identificar un sonido hablado, es necesario entender el cómo los seres humanos decodificamos los sonidos hablados.

MFCCs: en base a lo anterior, los MFCCs por lo tanto representan con presición este fin. Actualmente son muy utilizados para el reconocimiento del habla. 

A continuación se presenta el código necesario para obtener los MFCCs de la base de datos <a href="https://link.springer.com/article/10.1007/s10579-015-9324-5">KALAKA3</a>

## Pasos para la creación de MFCCs
1. Organización del dataset
2. Lectura y obtención de MFCCs
    - i. Obtención de las señales en Numpy Arrays
    - ii. Separar la señal de audio en frames.
    - iii. Cálculo del periodograma estimado de la potencia de la señal
    - iv. Aplicar el filtro mel, sumar la energía en cada filtro
    - v. Obtención del logaritmo de las energías del filterbank
    - vi. Obtención de la DCT
    - vii.Guardar los coeficientes (2-13) de la DCT, descartar lo demás
3. Creación de la red de entrenamiento
4. Análisis de resultados


## 1. Organización del dataset
En primer lugar es necesario organizar el dataset de la siguiente manera:
- KALAKA3
    - DEV_KALAKA3
    - EVAL_KALAKA3\eval\data
    - TRAIN_KALAKA3\data
        - Basque
            - clean
            - noisy
        - ....
            ....
        - Spanish
            - clean
            - noisy
                - 0a7707a3.wav
                - ...
                - ff5e57c3.wav
                
### Carga de librerías necesarias

In [135]:
# This file includes routines for basic signal processing including framing and computing power spectra.
# Author: James Lyons 2012
import decimal
import numpy
import math
import logging
import os
from scipy.fftpack import dct
from scipy.io.wavfile import read as readwav
import sys

## Especificación de Paths
Esta sección es muy importante para que el método de creación de los MFCCs pueda iterar de manera correcta, tanto para la lecutra de los audios, como para el grabado de los MFCCs.

In [6]:

SOURCE = 'KALAKA3' 
OUTPUT = 'MFCCS'


In [140]:
# This file includes routines for basic signal processing including framing and computing power spectra.
# Author: James Lyons 2012
def round_half_up(number):
    return int(decimal.Decimal(number).quantize(decimal.Decimal('1'), rounding=decimal.ROUND_HALF_UP))


def rolling_window(a, window, step=1):
    # http://ellisvalentiner.com/post/2017-03-21-np-strides-trick
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return numpy.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)[::step]


def framesig(sig, frame_len, frame_step, winfunc=lambda x: numpy.ones((x,)), stride_trick=True):
    """Frame a signal into overlapping frames.
    :param sig: the audio signal to frame.
    :param frame_len: length of each frame measured in samples.
    :param frame_step: number of samples after the start of the previous frame that the next frame should begin.
    :param winfunc: the analysis window to apply to each frame. By default no window is applied.
    :param stride_trick: use stride trick to compute the rolling window and window multiplication faster
    :returns: an array of frames. Size is NUMFRAMES by frame_len.
    """
    slen = len(sig)
    frame_len = int(round_half_up(frame_len))
    frame_step = int(round_half_up(frame_step))
    if slen <= frame_len:
        numframes = 1
    else:
        numframes = 1 + int(math.ceil((1.0 * slen - frame_len) / frame_step))

    padlen = int((numframes - 1) * frame_step + frame_len)

    zeros = numpy.zeros((padlen - slen,))
    padsignal = numpy.concatenate((sig, zeros))
    if stride_trick:
        win = winfunc(frame_len)
        frames = rolling_window(padsignal, window=frame_len, step=frame_step)
    else:
        indices = numpy.tile(numpy.arange(0, frame_len), (numframes, 1)) + numpy.tile(
            numpy.arange(0, numframes * frame_step, frame_step), (frame_len, 1)).T
        indices = numpy.array(indices, dtype=numpy.int32)
        frames = padsignal[indices]
        win = numpy.tile(winfunc(frame_len), (numframes, 1))

    return frames * win


def deframesig(frames, siglen, frame_len, frame_step, winfunc=lambda x: numpy.ones((x,))):
    """Does overlap-add procedure to undo the action of framesig.
    :param frames: the array of frames.
    :param siglen: the length of the desired signal, use 0 if unknown. Output will be truncated to siglen samples.
    :param frame_len: length of each frame measured in samples.
    :param frame_step: number of samples after the start of the previous frame that the next frame should begin.
    :param winfunc: the analysis window to apply to each frame. By default no window is applied.
    :returns: a 1-D signal.
    """
    frame_len = round_half_up(frame_len)
    frame_step = round_half_up(frame_step)
    numframes = numpy.shape(frames)[0]
    assert numpy.shape(frames)[1] == frame_len, '"frames" matrix is wrong size, 2nd dim is not equal to frame_len'

    indices = numpy.tile(numpy.arange(0, frame_len), (numframes, 1)) + numpy.tile(
        numpy.arange(0, numframes * frame_step, frame_step), (frame_len, 1)).T
    indices = numpy.array(indices, dtype=numpy.int32)
    padlen = (numframes - 1) * frame_step + frame_len

    if siglen <= 0: siglen = padlen

    rec_signal = numpy.zeros((padlen,))
    window_correction = numpy.zeros((padlen,))
    win = winfunc(frame_len)

    for i in range(0, numframes):
        window_correction[indices[i, :]] = window_correction[
                                               indices[i, :]] + win + 1e-15  # add a little bit so it is never zero
        rec_signal[indices[i, :]] = rec_signal[indices[i, :]] + frames[i, :]

    rec_signal = rec_signal / window_correction
    return rec_signal[0:siglen]


def magspec(frames, NFFT):
    """Compute the magnitude spectrum of each frame in frames. If frames is an NxD matrix, output will be Nx(NFFT/2+1).
    :param frames: the array of frames. Each row is a frame.
    :param NFFT: the FFT length to use. If NFFT > frame_len, the frames are zero-padded.
    :returns: If frames is an NxD matrix, output will be Nx(NFFT/2+1). Each row will be the magnitude spectrum of the corresponding frame.
    """
    if numpy.shape(frames)[1] > NFFT:
        logging.warn(
            'frame length (%d) is greater than FFT size (%d), frame will be truncated. Increase NFFT to avoid.',
            numpy.shape(frames)[1], NFFT)
    complex_spec = numpy.fft.rfft(frames, NFFT)
    return numpy.absolute(complex_spec)


def powspec(frames, NFFT):
    """Compute the power spectrum of each frame in frames. If frames is an NxD matrix, output will be Nx(NFFT/2+1).
    :param frames: the array of frames. Each row is a frame.
    :param NFFT: the FFT length to use. If NFFT > frame_len, the frames are zero-padded.
    :returns: If frames is an NxD matrix, output will be Nx(NFFT/2+1). Each row will be the power spectrum of the corresponding frame.
    """
    return 1.0 / NFFT * numpy.square(magspec(frames, NFFT))


def logpowspec(frames, NFFT, norm=1):
    """Compute the log power spectrum of each frame in frames. If frames is an NxD matrix, output will be Nx(NFFT/2+1).
    :param frames: the array of frames. Each row is a frame.
    :param NFFT: the FFT length to use. If NFFT > frame_len, the frames are zero-padded.
    :param norm: If norm=1, the log power spectrum is normalised so that the max value (across all frames) is 0.
    :returns: If frames is an NxD matrix, output will be Nx(NFFT/2+1). Each row will be the log power spectrum of the corresponding frame.
    """
    ps = powspec(frames, NFFT);
    ps[ps <= 1e-30] = 1e-30
    lps = 10 * numpy.log10(ps)
    if norm:
        return lps - numpy.max(lps)
    else:
        return lps


def preemphasis(signal, coeff=0.95):
    """perform preemphasis on the input signal.
    :param signal: The signal to filter.
    :param coeff: The preemphasis coefficient. 0 is no filter, default is 0.95.
    :returns: the filtered signal.
    """
    return numpy.append(signal[0], signal[1:] - coeff * signal[:-1])

In [151]:
# calculate filterbank features. Provides e.g. fbank and mfcc features for use in ASR applications
# Author: James Lyons 2012

def calculate_nfft(samplerate, winlen):
    """Calculates the FFT size as a power of two greater than or equal to
    the number of samples in a single window length.
    
    Having an FFT less than the window length loses precision by dropping
    many of the samples; a longer FFT than the window allows zero-padding
    of the FFT buffer which is neutral in terms of frequency domain conversion.
    :param samplerate: The sample rate of the signal we are working with, in Hz.
    :param winlen: The length of the analysis window in seconds.
    """
    window_length_samples = winlen * samplerate
    nfft = 1
    while nfft < window_length_samples:
        nfft *= 2
    return nfft

def mfcc(signal,samplerate=16000,winlen=0.025,winstep=0.01,numcep=13,
         nfilt=26,nfft=None,lowfreq=0,highfreq=None,preemph=0.97,ceplifter=22,appendEnergy=True,
         winfunc=lambda x:numpy.ones((x,))):
    """Compute MFCC features from an audio signal.
    :param signal: the audio signal from which to compute features. Should be an N*1 array
    :param samplerate: the sample rate of the signal we are working with, in Hz.
    :param winlen: the length of the analysis window in seconds. Default is 0.025s (25 milliseconds)
    :param winstep: the step between successive windows in seconds. Default is 0.01s (10 milliseconds)
    :param numcep: the number of cepstrum to return, default 13
    :param nfilt: the number of filters in the filterbank, default 26.
    :param nfft: the FFT size. Default is None, which uses the calculate_nfft function to choose the smallest size that does not drop sample data.
    :param lowfreq: lowest band edge of mel filters. In Hz, default is 0.
    :param highfreq: highest band edge of mel filters. In Hz, default is samplerate/2
    :param preemph: apply preemphasis filter with preemph as coefficient. 0 is no filter. Default is 0.97.
    :param ceplifter: apply a lifter to final cepstral coefficients. 0 is no lifter. Default is 22.
    :param appendEnergy: if this is true, the zeroth cepstral coefficient is replaced with the log of the total frame energy.
    :param winfunc: the analysis window to apply to each frame. By default no window is applied. You can use numpy window functions here e.g. winfunc=numpy.hamming
    :returns: A numpy array of size (NUMFRAMES by numcep) containing features. Each row holds 1 feature vector.
    """
    nfft = nfft or calculate_nfft(samplerate, winlen)
    feat,energy = fbank(signal,samplerate,winlen,winstep,nfilt,nfft,lowfreq,highfreq,preemph,winfunc)
    feat = numpy.log(feat)
    feat = dct(feat, type=2, axis=1, norm='ortho')[:,:numcep]
    feat = lifter(feat,ceplifter)
    if appendEnergy: feat[:,0] = numpy.log(energy) # replace first cepstral coefficient with log of frame energy
    return feat

def fbank(signal,samplerate=16000,winlen=0.025,winstep=0.01,
          nfilt=26,nfft=512,lowfreq=0,highfreq=None,preemph=0.97,
          winfunc=lambda x:numpy.ones((x,))):
    """Compute Mel-filterbank energy features from an audio signal.
    :param signal: the audio signal from which to compute features. Should be an N*1 array
    :param samplerate: the sample rate of the signal we are working with, in Hz.
    :param winlen: the length of the analysis window in seconds. Default is 0.025s (25 milliseconds)
    :param winstep: the step between successive windows in seconds. Default is 0.01s (10 milliseconds)
    :param nfilt: the number of filters in the filterbank, default 26.
    :param nfft: the FFT size. Default is 512.
    :param lowfreq: lowest band edge of mel filters. In Hz, default is 0.
    :param highfreq: highest band edge of mel filters. In Hz, default is samplerate/2
    :param preemph: apply preemphasis filter with preemph as coefficient. 0 is no filter. Default is 0.97.
    :param winfunc: the analysis window to apply to each frame. By default no window is applied. You can use numpy window functions here e.g. winfunc=numpy.hamming
    :returns: 2 values. The first is a numpy array of size (NUMFRAMES by nfilt) containing features. Each row holds 1 feature vector. The
        second return value is the energy in each frame (total energy, unwindowed)
    """
    highfreq= highfreq or samplerate/2
    signal = preemphasis(signal,preemph)
    frames = framesig(signal, winlen*samplerate, winstep*samplerate, winfunc)
    pspec = powspec(frames,nfft)
    energy = numpy.sum(pspec,1) # this stores the total energy in each frame
    energy = numpy.where(energy == 0,numpy.finfo(float).eps,energy) # if energy is zero, we get problems with log

    fb = get_filterbanks(nfilt,nfft,samplerate,lowfreq,highfreq)
    feat = numpy.dot(pspec,fb.T) # compute the filterbank energies
    feat = numpy.where(feat == 0,numpy.finfo(float).eps,feat) # if feat is zero, we get problems with log

    return feat,energy

def logfbank(signal,samplerate=16000,winlen=0.025,winstep=0.01,
             nfilt=26,nfft=512,lowfreq=0,highfreq=None,preemph=0.97,
             winfunc=lambda x:numpy.ones((x,))):
    """Compute log Mel-filterbank energy features from an audio signal.
    :param signal: the audio signal from which to compute features. Should be an N*1 array
    :param samplerate: the sample rate of the signal we are working with, in Hz.
    :param winlen: the length of the analysis window in seconds. Default is 0.025s (25 milliseconds)
    :param winstep: the step between successive windows in seconds. Default is 0.01s (10 milliseconds)
    :param nfilt: the number of filters in the filterbank, default 26.
    :param nfft: the FFT size. Default is 512.
    :param lowfreq: lowest band edge of mel filters. In Hz, default is 0.
    :param highfreq: highest band edge of mel filters. In Hz, default is samplerate/2
    :param preemph: apply preemphasis filter with preemph as coefficient. 0 is no filter. Default is 0.97.
    :param winfunc: the analysis window to apply to each frame. By default no window is applied. You can use numpy window functions here e.g. winfunc=numpy.hamming
    :returns: A numpy array of size (NUMFRAMES by nfilt) containing features. Each row holds 1 feature vector.
    """
    feat,energy = fbank(signal,samplerate,winlen,winstep,nfilt,nfft,lowfreq,highfreq,preemph,winfunc)
    return numpy.log(feat)

def ssc(signal,samplerate=16000,winlen=0.025,winstep=0.01,
        nfilt=26,nfft=512,lowfreq=0,highfreq=None,preemph=0.97,
        winfunc=lambda x:numpy.ones((x,))):
    """Compute Spectral Subband Centroid features from an audio signal.
    :param signal: the audio signal from which to compute features. Should be an N*1 array
    :param samplerate: the sample rate of the signal we are working with, in Hz.
    :param winlen: the length of the analysis window in seconds. Default is 0.025s (25 milliseconds)
    :param winstep: the step between successive windows in seconds. Default is 0.01s (10 milliseconds)
    :param nfilt: the number of filters in the filterbank, default 26.
    :param nfft: the FFT size. Default is 512.
    :param lowfreq: lowest band edge of mel filters. In Hz, default is 0.
    :param highfreq: highest band edge of mel filters. In Hz, default is samplerate/2
    :param preemph: apply preemphasis filter with preemph as coefficient. 0 is no filter. Default is 0.97.
    :param winfunc: the analysis window to apply to each frame. By default no window is applied. You can use numpy window functions here e.g. winfunc=numpy.hamming
    :returns: A numpy array of size (NUMFRAMES by nfilt) containing features. Each row holds 1 feature vector.
    """
    highfreq= highfreq or samplerate/2
    signal = preemphasis(signal,preemph)
    frames = framesig(signal, winlen*samplerate, winstep*samplerate, winfunc)
    pspec = powspec(frames,nfft)
    pspec = numpy.where(pspec == 0,numpy.finfo(float).eps,pspec) # if things are all zeros we get problems

    fb = get_filterbanks(nfilt,nfft,samplerate,lowfreq,highfreq)
    feat = numpy.dot(pspec,fb.T) # compute the filterbank energies
    R = numpy.tile(numpy.linspace(1,samplerate/2,numpy.size(pspec,1)),(numpy.size(pspec,0),1))

    return numpy.dot(pspec*R,fb.T) / feat

def hz2mel(hz):
    """Convert a value in Hertz to Mels
    :param hz: a value in Hz. This can also be a numpy array, conversion proceeds element-wise.
    :returns: a value in Mels. If an array was passed in, an identical sized array is returned.
    """
    return 2595 * numpy.log10(1+hz/700.)

def mel2hz(mel):
    """Convert a value in Mels to Hertz
    :param mel: a value in Mels. This can also be a numpy array, conversion proceeds element-wise.
    :returns: a value in Hertz. If an array was passed in, an identical sized array is returned.
    """
    return 700*(10**(mel/2595.0)-1)

def get_filterbanks(nfilt=20,nfft=512,samplerate=16000,lowfreq=0,highfreq=None):
    """Compute a Mel-filterbank. The filters are stored in the rows, the columns correspond
    to fft bins. The filters are returned as an array of size nfilt * (nfft/2 + 1)
    :param nfilt: the number of filters in the filterbank, default 20.
    :param nfft: the FFT size. Default is 512.
    :param samplerate: the sample rate of the signal we are working with, in Hz. Affects mel spacing.
    :param lowfreq: lowest band edge of mel filters, default 0 Hz
    :param highfreq: highest band edge of mel filters, default samplerate/2
    :returns: A numpy array of size nfilt * (nfft/2 + 1) containing filterbank. Each row holds 1 filter.
    """
    highfreq= highfreq or samplerate/2
    assert highfreq <= samplerate/2, "highfreq is greater than samplerate/2"

    # compute points evenly spaced in mels
    lowmel = hz2mel(lowfreq)
    highmel = hz2mel(highfreq)
    melpoints = numpy.linspace(lowmel,highmel,nfilt+2)
    # our points are in Hz, but we use fft bins, so we have to convert
    #  from Hz to fft bin number
    bin = numpy.floor((nfft+1)*mel2hz(melpoints)/samplerate)

    fbank = numpy.zeros([nfilt,nfft//2+1])
    for j in range(0,nfilt):
        for i in range(int(bin[j]), int(bin[j+1])):
            fbank[j,i] = (i - bin[j]) / (bin[j+1]-bin[j])
        for i in range(int(bin[j+1]), int(bin[j+2])):
            fbank[j,i] = (bin[j+2]-i) / (bin[j+2]-bin[j+1])
    return fbank

def lifter(cepstra, L=22):
    """Apply a cepstral lifter the the matrix of cepstra. This has the effect of increasing the
    magnitude of the high frequency DCT coeffs.
    :param cepstra: the matrix of mel-cepstra, will be numframes * numcep in size.
    :param L: the liftering coefficient to use. Default is 22. L <= 0 disables lifter.
    """
    if L > 0:
        nframes,ncoeff = numpy.shape(cepstra)
        n = numpy.arange(ncoeff)
        lift = 1 + (L/2.)*numpy.sin(numpy.pi*n/L)
        return lift*cepstra
    else:
        # values of L <= 0, do nothing
        return cepstra

def delta(feat, N):
    """Compute delta features from a feature vector sequence.
    :param feat: A numpy array of size (NUMFRAMES by number of features) containing features. Each row holds 1 feature vector.
    :param N: For each frame, calculate delta features based on preceding and following N frames
    :returns: A numpy array of size (NUMFRAMES by number of features) containing delta features. Each row holds 1 delta feature vector.
    """
    if N < 1:
        raise ValueError('N must be an integer >= 1')
    NUMFRAMES = len(feat)
    denominator = 2 * sum([i**2 for i in range(1, N+1)])
    delta_feat = numpy.empty_like(feat)
    padded = numpy.pad(feat, ((N, N), (0, 0)), mode='edge')   # padded version of feat
    for t in range(NUMFRAMES):
        delta_feat[t] = numpy.dot(numpy.arange(-N, N+1), padded[t : t+2*N+1]) / denominator   # [t : t+2*N+1] == [(N+t)-N : (N+t)+N+1]
    return delta_feat

In [67]:
import matplotlib.pyplot as plt
def plotMfcc(filter_banks):
    plt.subplot(312)
    filter_banks -= (numpy.mean(filter_banks,axis=0) + 1e-8)
    plt.imshow(filter_banks.T, cmap=plt.cm.jet, aspect='auto')
    plt.xticks(numpy.arange(0, (filter_banks.T).shape[1],
    int((filter_banks.T).shape[1] / 4)),
    ['0s', '0.5s', '1s', '1.5s','2.5s','3s','3.5'])
    ax = plt.gca()
    ax.invert_yaxis()
    plt.title('the spectrum image')

### Creación de los MFCCs
El siguiente bucle lee y crea los MFCCs desde el SOURCE especificado, luego crea una carpeta con el nombre OUTPUT con la misma estructura del dataset de los archivos de audio (WAVs).

In [ ]:
i=0
for root, dirs, files in os.walk( SOURCE ):

    for archivo in files:
        if archivo.endswith('wav'):
            output_dir = root.replace( SOURCE, OUTPUT   )
            output_filename = archivo.split('.')[0]
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            input_path = "{}/{}".format( root,archivo ) #DIRECCION DEL ARCHIVO WAV A PROCESAR
            output_path = "{}/{}.mfccs".format( output_dir, output_filename ) #DIRECCION DONDE SE VA A GUARDAR EL ARCHIVO DE MFCC
            print("{}: {} {}".format(i,input_path,output_path))#SE IMPRIME ORIGEN Y DESTINO 
            i+=1
            _, input= readwav(input_path) #SE LEE EL ARCHIVO DE AUDIO Y SE GUARDA COMO UN ARRAY DE NUMPY

            output = mfcc( input )
            
            print(output.shape)
            output.tofile( output_path )

## Lectura de los MFCCs para su procesamiento
A continuacion se elaborara un metodo para compactar los MFCCs en un archivo csv, que es un poco mas versatil que guardarlo por separado

### Construccion del training

In [285]:
# make a new folder in this directory to save our results in

TRAINDIR="MFCCS/TRAIN_KALAKA3/data"
X_MFCCs=[]
Y_MFCCs=[]
if not os.path.exists(TRAINDIR):
    print("No se encuentra ningun MFCC")
    sys.exit()
# get MFCCs for every .wav file in our specified directory 
#especificar el numero de wavs a leer
numWavs=2
#se almacenará en esta variable los mfccs y sus etiquetas
for idioma in os.listdir(TRAINDIR):
  
    sourceMFCC=TRAINDIR+"/"+idioma
    print(idioma)

    for root, dirs, files in os.walk( sourceMFCC):
        for archivo in files:
            if archivo.endswith('mfccs'): # only get MFCCs 
                mfccFile = root + "/" + os.path.splitext(archivo)[0] + ".mfccs"
                #print(mfccFile)
                #Establezco el label del MFCC 
                mfcc=numpy.fromfile(mfccFile)
                mfccFrames=mfcc.reshape(int(mfcc.shape[0]/13),13)
                for frame in mfccFrames:
                    X_MFCCs.append(frame)#almaceno el MFCC
                    Y_MFCCs.append(idioma)#almaceno el idioma al que pertenece el MFCC
    

Basque
Catalan
English
Galician
Portuguese
Spanish


In [283]:
# make a new folder in this directory to save our results in

TESTDIR="MFCCS/TEST_KALAKA3/data"
X_TEST_MFCCs=[]
Y_TEST_MFCCs=[]
if not os.path.exists(TESTDIR):
    print("No se encuentra ningun MFCC")
    sys.exit()
# get MFCCs for every .wav file in our specified directory 
#se almacenará en esta variable los mfccs y sus etiquetas
for idioma in os.listdir(TESTDIR):
  
    sourceMFCC=TESTDIR+"/"+idioma
    print(idioma)

    for root, dirs, files in os.walk( sourceMFCC):
        for archivo in files:
            if archivo.endswith('mfccs'): # only get MFCCs 
                mfccFile = root + "/" + os.path.splitext(archivo)[0] + ".mfccs"
                #print(mfccFile)
                #Establezco el label del MFCC 
                mfcc=numpy.fromfile(mfccFile)
                #print(mfcc.shape)
                mfccFrames=mfcc.reshape(int(mfcc.shape[0]/13),13)
                #print(mfccFrames.shape)
                #print(mfccFrames[0])
                for frame in mfccFrames:
                    X_TEST_MFCCs.append(frame)#almaceno el MFCC
                    Y_TEST_MFCCs.append(idioma)#almaceno el idioma al que pertenece el MFCC
    

Basque
Catalan
English
Galician
Portuguese
Spanish


In [222]:
#Obteniendo un mfcc para pruebas
mfccTmp=numpy.fromfile("MFCCS/TRAIN_KALAKA3/data/Basque\clean/00248e27.mfccs")

In [183]:
#!/usr/bin/env python
#Metodos a utilizar
#mfcc
#delta
#logfbank
#Obteniendo el mfcc de un wav para compararlo con el Mfcc anterior
import scipy.io.wavfile as wav
(rate,sig) = wav.read("KALAKA3/TRAIN_KALAKA3/data\Catalan\clean/1a8b426c.wav")
print(rate)
mfcc_feat = mfcc(sig,rate)
d_mfcc_feat = delta(mfcc_feat, 2)
fbank_feat = logfbank(sig,rate)

16000


In [1]:
mfccFrames=mfccTmp.reshape(int(mfccTmp.shape[0]/13),13) 
print(mfccTmp.shape)
print(mfccFrames.shape)
plotMfcc(mfccFrames)

NameError: name 'mfccTmp' is not defined

## Reconicimiento Automático del habla (ASR)
### Carga de los mfccs
                              

In [287]:
#Se convierten a arrays las listas obtenidas
X_MFCCs=numpy.asarray(X_MFCCs)
Y_MFCCs=numpy.asarray(Y_MFCCs)
X_TEST_MFCCs=numpy.asarray(X_TEST_MFCCs)
Y_TEST_MFCCs=numpy.asarray(Y_TEST_MFCCs)


In [312]:
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
import tensorflow as tf
from keras.models import Sequential
#from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from deepacoustics.tf.util import get_class_weights


with tf.Session() as sess0:
    assert not tf.executing_eagerly()
    model = Sequential()

    model.add(Dense(32, input_shape=X_MFCCs.shape[1:], activation='tanh'))
    model.add(Dense(64, activation='tanh'))
    model.add(Dense(128, activation='tanh'))
    
    #model.add(Flatten())
    #model.add(Dense(256, activation='relu'))

    model.add(Dense(30))
    model.add(Activation('sigmoid'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    #Adam, an algorithm for first-order gradient-based optimization of stochastic objective functions, based on adaptive estimates of lower-order moments.
    model.summary()
  # history = model.fit(x=X_train_array, y=y_train_array, epochs=5, verbose=1, validation_split = 0.33, shuffle=True, class_weight=get_class_weights(pd.Series((list(set(labels))),dtype='category').cat.codes.values),batch_size=batch_size) 
    history = model.fit(x=X_MFCCs, y=Y_MFCCs, epochs=25, verbose=1, validation_split = 0.1, shuffle=True, class_weight=get_class_weights(pd.Series(Y_MFCCs,dtype='category').cat.codes.values),batch_size=128) 
    
    model_evaluation = model.evaluate(x=X_TEST_MFCCs, y=Y_TEST_MFCCs, batch_size=None, verbose=1)

    prediction = model.predict(X_TEST_MFCCs, batch_size = 128, verbose = 1)
    
    #april_tst = model.predict(mfcc_april_test, batch_size = 128, verbose = 1)

    sess0.close()

ModuleNotFoundError: No module named 'deepacoustics'

In [313]:
num_labels = Y_MFCCs.shape[0]

def build_model_graph(input_shape=(13,)):
    model = Sequential()
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    # Compile the model
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')    
    modelmodel = build_model_graph()
    return modelmodel



In [ ]:
filter_size = build_model_graph(input_shape=(13,))

Exception ignored in: <function BaseSession.__del__ at 0x000001EA41E4D048>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 738, in __del__
    tf_session.TF_DeleteSession(self._session)
KeyboardInterrupt: 
Exception ignored in: <function BaseSession._Callable.__del__ at 0x000001EA41E4DAE8>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.UnimplementedError: ReleaseCallable is not supported for this session.


In [304]:
# Display model architecture summary 
model.summary()# Calculate pre-training accuracy 
score = model.evaluate(X_TEST_MFCCs, Y_TEST_MFCCs, verbose=2)
accuracy = 100*score[1]

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 32)                448       
_________________________________________________________________
dense_24 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_25 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_26 (Dense)             (None, 30)                3870      
_________________________________________________________________
activation_4 (Activation)    (None, 30)                0         
Total params: 14,750
Trainable params: 14,750
Non-trainable params: 0
_________________________________________________________________


ValueError: Error when checking target: expected activation_4 to have shape (30,) but got array with shape (1,)

## Referencias
- <a href="http://practicalcryptography.com/miscellaneous/machine-learning/guide-mel-frequency-cepstral-coefficients-mfccs/">Mel Frequency Cepstral Coefficient (MFCC) tutorial</a> 
- <a href="https://www.researchgate.net/publication/330477843_A_Mel-Filterbank_and_MFCC-based_Neural_Network_Approach_to_Train_the_Houston_Toad_Call_Detection _System_Design">A Mel-Filterbank and MFCC-based Neural Network Approach to Train the Houston Toad Call Detection System Design </a> 
- <a href="https://towardsdatascience.com/speech-recognition-analysis-f03ff9ce78e9">Speech Recognition Analysis</a> 
- <a href="https://github.com/rctatman/getMFCCs/blob/master/getMFCCs.py">getMFCCs</a> 
- <a href="https://towardsdatascience.com/how-to-apply-machine-learning-and-deep-learning-methods-to-audio-analysis-615e286fcbbc">How to apply machine learning and deep learning methods to audio analysis</a>




